### Data Preprocessing

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA 
from matplotlib import pyplot as plt
from sklearn.decomposition import SparsePCA
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


In [ ]:
# Load the data
path = 'robusted_merged_data.csv'
data = pd.read_csv(path)
data

In [ ]:
# Check the NAs in each column
for col in data.columns:
    count = data[col].isna().sum()
    print(f"{col}: {count}")

In [ ]:
# Adjust the content of the final rank column, and keep only two ranks
data = data.drop(columns = ['Nan'])
keep = ['Brigadier General', 'Major General']
data = data[data['final rank'].isin(keep)].copy()
data

In [ ]:
# Define the outcome variable and convert it to a binary.
data['rank_code'] = data['final rank'].map({'Brigadier General': 0, 'Major General': 1})
# Define columns of medals
medal_columns = [c for c in data.select_dtypes(include = 'int64').columns if c not in ['name', 'final rank', 'Number of Years Served', 'average_rating', 'n_valid_ratings','rank_code']]
# Double Check
print(medal_columns)


In [ ]:
# To repsect the original paper, we convert the average_rating into a cardinal order.
data['rating'] = data['average_rating']
data = data.drop(columns = ['rating'])
data

In [ ]:
data.describe()

In [ ]:
data.head()

### First, let's consider the accuracy of models under the normal logistic, probit, and GLM regressions.

In [ ]:
predictors = ['rating']
y = data['rank_code']
X_constant = sm.add_constant(data[predictors].astype(float))
logit = sm.Logit(y, X_constant)
result = logit.fit(disp=False)

result.summary()

In [ ]:
predictors = ['rating']
y = data['rank_code']
X_constant = sm.add_constant(data[predictors].astype(float))
probit = sm.Probit(y, X_constant)
result = probit.fit(disp=False)

result.summary()

In [ ]:
# One-time Logit
data['num_medals'] = data[medal_columns].sum(axis = 1)
predictors = ['Number of Years Served', 'rating', 'num_medals']

X_constant = sm.add_constant(data[predictors].astype(float))
y = data['rank_code']

logit = sm.Logit(y, X_constant)
result = logit.fit(disp=False)

result.summary()

In [ ]:
data.describe()

In [ ]:
# One-time Probit
data['num_medals'] = data[medal_columns].sum(axis = 1)
predictors = ['Number of Years Served', 'rating', 'num_medals']

X_constant = sm.add_constant(data[predictors].astype(float))
y = data['rank_code']

probit = sm.Probit(y, X_constant)
result = probit.fit(disp=False)

result.summary()

In [ ]:
# Now let's see one-time logit without rating.
data['num_medals'] = data[medal_columns].sum(axis = 1)
predictors = ['Number of Years Served', 'num_medals']

X_constant = sm.add_constant(data[predictors].astype(float))
y = data['rank_code']

logit = sm.Logit(y, X_constant)
result = logit.fit(disp=False)

result.summary()

In [ ]:
# One-time Probit without rating.
data['num_medals'] = data[medal_columns].sum(axis = 1)
predictors = ['Number of Years Served', 'num_medals']

X_constant = sm.add_constant(data[predictors].astype(float))
y = data['rank_code']

probit = sm.Probit(y, X_constant)
result = probit.fit(disp=False)

result.summary()

In [ ]:
# Bootstrpped Logit and its performance.
X = data[predictors].astype(float)
y = data['rank_code']

n_bootstrap = 200
boot_accuracies = []
boot_coefs= []

for i in range(n_bootstrap):
    X_bs, y_bs = resample(X, y, replace=True)

    # Skip if only one class is sampled (perfect class imbalance)
    if len(np.unique(y_bs)) < 2:
        continue

    # Scale within bootstrap
    scaler = StandardScaler()
    X_bs_scaled = scaler.fit_transform(X_bs)
    X_bs_scaled = sm.add_constant(X_bs_scaled)

    try:
        model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        boot_coefs.append(model.params.values)
    except Exception as e:
        print(f"Iteration {i} failed: {e}")
        continue

# Wrap results
if len(boot_accuracies) > 0:
    acc_summary = pd.Series(boot_accuracies).describe(percentiles=[.05, .25, .5, .75, .95])
    coef_df = pd.DataFrame(boot_coefs, columns=['Intercept'] + predictors)

    print("✅ Bootstrapped Accuracy Summary:")
    print(acc_summary)

    print("\n✅ Bootstrap Coefficient Means:")
    print(coef_df.mean())

    print("\n✅ Bootstrap Coefficient Std Deviations:")
    print(coef_df.std())
else:
    print("❌ All bootstrap iterations failed. Consider regularization or removing strong predictors.")

In [ ]:
# Bootstrpped Probit and its performance.
X = data[predictors].astype(float)
y = data['rank_code']

n_bootstrap = 200
boot_accuracies = []
boot_coefs= []

for i in range(n_bootstrap):
    X_bs, y_bs = resample(X, y, replace=True)

    # Skip if only one class is sampled (perfect class imbalance)
    if len(np.unique(y_bs)) < 2:
        continue

    # Scale within bootstrap
    scaler = StandardScaler()
    X_bs_scaled = scaler.fit_transform(X_bs)
    X_bs_scaled = sm.add_constant(X_bs_scaled)

    try:
        model = sm.Probit(y_bs, X_bs_scaled).fit(disp=False)
        y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        boot_coefs.append(model.params.values)
    except Exception as e:
        print(f"Iteration {i} failed: {e}")
        continue

# Wrap results
if len(boot_accuracies) > 0:
    acc_summary = pd.Series(boot_accuracies).describe(percentiles=[.05, .25, .5, .75, .95])
    coef_df = pd.DataFrame(boot_coefs, columns=['Intercept'] + predictors)

    print("✅ Bootstrapped Accuracy Summary:")
    print(acc_summary)

    print("\n✅ Bootstrap Coefficient Means:")
    print(coef_df.mean())

    print("\n✅ Bootstrap Coefficient Std Deviations:")
    print(coef_df.std())
else:
    print("❌ All bootstrap iterations failed. Consider regularization or removing strong predictors.")

In [ ]:
# Bootstrpped Logit and its performance without rating.
X = data[predictors].astype(float)
y = data['rank_code']

n_bootstrap = 200
boot_accuracies = []
boot_coefs= []

for i in range(n_bootstrap):
    X_bs, y_bs = resample(X, y, replace=True)

    # Skip if only one class is sampled (perfect class imbalance)
    if len(np.unique(y_bs)) < 2:
        continue

    # Scale within bootstrap
    scaler = StandardScaler()
    X_bs_scaled = scaler.fit_transform(X_bs)
    X_bs_scaled = sm.add_constant(X_bs_scaled)

    try:
        model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        boot_coefs.append(model.params.values)
    except Exception as e:
        print(f"Iteration {i} failed: {e}")
        continue

# Wrap results
if len(boot_accuracies) > 0:
    acc_summary = pd.Series(boot_accuracies).describe(percentiles=[.05, .25, .5, .75, .95])
    coef_df = pd.DataFrame(boot_coefs, columns=['Intercept'] + predictors)

    print("✅ Bootstrapped Accuracy Summary:")
    print(acc_summary)

    print("\n✅ Bootstrap Coefficient Means:")
    print(coef_df.mean())

    print("\n✅ Bootstrap Coefficient Std Deviations:")
    print(coef_df.std())
else:
    print("❌ All bootstrap iterations failed. Consider regularization or removing strong predictors.")

In [ ]:
# Bootstrpped Probit and its performance without rating.
X = data[predictors].astype(float)
y = data['rank_code']

n_bootstrap = 200
boot_accuracies = []
boot_coefs= []

for i in range(n_bootstrap):
    X_bs, y_bs = resample(X, y, replace=True)

    # Skip if only one class is sampled (perfect class imbalance)
    if len(np.unique(y_bs)) < 2:
        continue

    # Scale within bootstrap
    scaler = StandardScaler()
    X_bs_scaled = scaler.fit_transform(X_bs)
    X_bs_scaled = sm.add_constant(X_bs_scaled)

    try:
        model = sm.Probit(y_bs, X_bs_scaled).fit(disp=False)
        y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        boot_coefs.append(model.params.values)
    except Exception as e:
        print(f"Iteration {i} failed: {e}")
        continue

# Wrap results
if len(boot_accuracies) > 0:
    acc_summary = pd.Series(boot_accuracies).describe(percentiles=[.05, .25, .5, .75, .95])
    coef_df = pd.DataFrame(boot_coefs, columns=['Intercept'] + predictors)

    print("✅ Bootstrapped Accuracy Summary:")
    print(acc_summary)

    print("\n✅ Bootstrap Coefficient Means:")
    print(coef_df.mean())

    print("\n✅ Bootstrap Coefficient Std Deviations:")
    print(coef_df.std())
else:
    print("❌ All bootstrap iterations failed. Consider regularization or removing strong predictors.")

### Second, let's consider the accuracy under PCA with regularized Logit.

In [ ]:
# Updated function with automatic fallback to fit_regularized() when fit() fails

def run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.80, n_bootstrap=30, verbose=True):

    # Step 1: PCA
    X_medals = data[medal_columns].astype(float)
    pca_full = PCA()
    pca_full.fit(X_medals)
    explained_var = np.cumsum(pca_full.explained_variance_ratio_)
    n_components = np.argmax(explained_var >= threshold) + 1

    if verbose:
        print(f"Threshold: {threshold:.2f} → Retaining {n_components} components")

    pca = PCA(n_components=n_components)
    X_pca = pd.DataFrame(pca.fit_transform(X_medals), columns=[f"PC{i+1}" for i in range(n_components)])

    # Step 2: Combine with predictors
    base_vars = ['Number of Years Served', 'rating']
    X = pd.concat([data[base_vars].reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)
    y = data['rank_code'].astype(int)
    coef_names = ['const'] + base_vars + list(X_pca.columns)

    # Step 3: Bootstrap logit
    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.1, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "threshold": threshold,
            "n_components": n_components,
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None



In [ ]:
# Updated function without 'rating' as a predictor

def run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.80, n_bootstrap=30, verbose=True):
    from sklearn.decomposition import PCA
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import statsmodels.api as sm
    import numpy as np
    import pandas as pd

    # Step 1: PCA
    X_medals = data[medal_columns].astype(float)
    pca_full = PCA()
    pca_full.fit(X_medals)
    explained_var = np.cumsum(pca_full.explained_variance_ratio_)
    n_components = np.argmax(explained_var >= threshold) + 1

    if verbose:
        print(f"Threshold: {threshold:.2f} → Retaining {n_components} components")

    pca = PCA(n_components=n_components)
    X_pca = pd.DataFrame(pca.fit_transform(X_medals), columns=[f"PC{i+1}" for i in range(n_components)])

    # Step 2: Combine with predictors (exclude 'rating')
    base_vars = ['Number of Years Served']
    X = pd.concat([data[base_vars].reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)
    y = data['rank_code'].astype(int)
    coef_names = ['const'] + base_vars + list(X_pca.columns)

    # Step 3: Bootstrap logit
    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.1, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "threshold": threshold,
            "n_components": n_components,
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None


In [ ]:
result_60 = run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.60)
result_60

In [ ]:
result_norating60 = run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.60, n_bootstrap=30, verbose=True)
result_norating60

In [ ]:
result_70 = run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.70)
result_70

In [ ]:
result_norating70 = run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.70, n_bootstrap=30, verbose=True)
result_norating70

In [ ]:
result_80 = run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.80)
result_80

In [ ]:
result_norating80 = run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.80, n_bootstrap=30, verbose=True)
result_norating80

In [ ]:
result_90 = run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.90)
result_90

In [ ]:
result_norating90 = run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.90, n_bootstrap=30, verbose=True)
result_norating90

In [ ]:
result_95 = run_bootstrap_pca_model_with_coefs(data, medal_columns, threshold=0.95)
result_95

In [ ]:
result_norating95 = run_bootstrap_pca_model_without_rating(data, medal_columns, threshold=0.95, n_bootstrap=30, verbose=True)
result_norating95

In [ ]:
result_80["mean_accuracy"]

In [ ]:
result_80["coef_df"]

In [ ]:
coef_df = result_80["coef_df"]

# Mean and standard deviation
coef_mean = coef_df.mean()
coef_std = coef_df.std()

# 95% Confidence Interval
ci_95_lower = coef_mean - 1.96 * coef_std
ci_95_upper = coef_mean + 1.96 * coef_std

# Combine into a summary table
coef_summary = pd.DataFrame({
    "Mean": coef_mean,
    "Std": coef_std,
    "95% CI Lower": ci_95_lower,
    "95% CI Upper": ci_95_upper
})
print(coef_summary.round(3))


In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(coef_mean.index, coef_mean, yerr=1.96 * coef_std, capsize=4)
plt.xticks(rotation=45, ha='right')
plt.ylabel("Coefficient Estimate")
plt.title("Bootstrapped Coefficient Estimates (Mean ± 95% CI)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.boxplot(data=coef_df, orient="h")
plt.title("Distribution of Bootstrapped Coefficients")
plt.xlabel("Coefficient Value")
plt.tight_layout()
plt.show()

### Third, Let's consider the accuracy under Sparse PCA.

In [ ]:
# Define the Sparse PCA bootstrap pipeline with coefficient output

def run_bootstrap_sparse_pca_model_with_coefs(data, medal_columns, n_components=5, n_bootstrap=30, alpha=1.0, verbose=True):

    if verbose:
        print(f"Sparse PCA → Manually set to {n_components} components | alpha = {alpha}")

    # Step 1: Sparse PCA
    X_medals = data[medal_columns].astype(float)
    spca = SparsePCA(n_components=n_components, alpha=alpha, random_state=42)
    X_pca = pd.DataFrame(spca.fit_transform(X_medals), columns=[f"SPC{i+1}" for i in range(n_components)])

    # Step 2: Combine with predictors
    base_vars = ['Number of Years Served', 'rating']
    X = pd.concat([data[base_vars].reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)
    y = data['rank_code'].astype(int)
    coef_names = ['const'] + base_vars + list(X_pca.columns)

    # Step 3: Bootstrap logit
    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.1, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "n_components": n_components,
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None


In [ ]:
# Modified version of Sparse PCA bootstrap function that excludes "rating"

def run_bootstrap_sparse_pca_model_without_rating(data, medal_columns, n_components=5, n_bootstrap=30, alpha=1.0, verbose=True):
    from sklearn.decomposition import SparsePCA
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import statsmodels.api as sm
    import numpy as np
    import pandas as pd

    if verbose:
        print(f"Sparse PCA → Manually set to {n_components} components | alpha = {alpha}")

    # Step 1: Sparse PCA
    X_medals = data[medal_columns].astype(float)
    spca = SparsePCA(n_components=n_components, alpha=alpha, random_state=42)
    X_pca = pd.DataFrame(spca.fit_transform(X_medals), columns=[f"SPC{i+1}" for i in range(n_components)])

    # Step 2: Combine with predictors (exclude 'rating')
    base_vars = ['Number of Years Served']
    X = pd.concat([data[base_vars].reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)
    y = data['rank_code'].astype(int)
    coef_names = ['const'] + base_vars + list(X_pca.columns)

    # Step 3: Bootstrap logit
    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.1, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "n_components": n_components,
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None


In [ ]:
result_sparse5 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=5,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating5 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=5, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:

result_sparse6 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=6,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating6 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=6, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse7 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=7,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating7 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=7, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse8 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=8,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating8 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=8, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse9 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=9,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating9 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=9, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse10 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=10,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating10 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=10, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse11 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=11,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating11 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=11, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
result_sparse12 = run_bootstrap_sparse_pca_model_with_coefs(
    data, 
    medal_columns, 
    n_components=12,   
    alpha=1.0,        
    n_bootstrap=30
)

In [ ]:
result_spca_no_rating12 = run_bootstrap_sparse_pca_model_without_rating(
    data, 
    medal_columns, 
    n_components=12, 
    n_bootstrap=30, 
    alpha=1.0
)

In [ ]:
coef_df = result_sparse10["coef_df"]
summary = pd.DataFrame({
    "Mean": coef_df.mean(),
    "Std": coef_df.std(),
    "95% CI Lower": coef_df.mean() - 1.96 * coef_df.std(),
    "95% CI Upper": coef_df.mean() + 1.96 * coef_df.std()
})
print(summary.round(3))

### Fourth, let's consider the Manually Grouping Logistic.

In [ ]:
# Define medal tiers
tier1 = [
    'Medal of Honor', 'Distinguished Service Cross', 'Navy Cross', 'Silver Star', 'Bronze Star'
]

tier2 = [
    'Distinguished Service Medal', 'Legion of Merit', 'Purple Heart', 'Oak Leaf Cluster',
    'Public Welfare Medal'
]

tier3 = [
    'Civil War Campaign Medal', 'Indian Campaign Medal', 'Mexican Border Service Medal',
    'Philippine Campaign Medal', 'Philippine Congressional Medal',
    'Cuban Pacification Medal', 'Spanish Campaign Medal', 'Spanish War Service Medal',
    'World War I Victory Medal', 'War Merit Cross', 
]

tier4 = [
    'Companion Of The Order Of The Bath (United Kingdom)', 'Croix de Guerre',
    'Czechoslovak War Cross 1918', 'French Legion Of Honor (Commander)',
    'Grand Cordon of the Order of the Sacred Treasure', 'Honorary Knight Commande',
    'Legion of Honor', 'Order Of Leopold Ii (Belgium)', 'Order Of The Black Star (Commander)',
    'Order of La Solidaridad (Panama)', 'Order of Leopold', 'Order of Saints Maurice and Lazarus',
    'Order of St Michael and St George', 'Order of Wen Hu', 'Order of the Bath',
    'Order of the Crown', 'Order of the Dragon of Annam', 'Order of the Rising Sun',
    'Order of the Star of Africa'
]

# Fill NaNs with 0 to avoid issues with summing
data[tier1 + tier2 + tier3 + tier4] = data[tier1 + tier2 + tier3 + tier4].fillna(0)

# Create new columns for medal tier counts
data['num_medal_tier1'] = data[tier1].sum(axis=1)
data['num_medal_tier2'] = data[tier2].sum(axis=1)
data['num_medal_tier3'] = data[tier3].sum(axis=1)
data['num_medal_tier4'] = data[tier4].sum(axis=1)

data

In [ ]:
num1 = data['num_medal_tier1'].sum()
num2 = data['num_medal_tier2'].sum()
num3 = data['num_medal_tier3'].sum()
num4 = data['num_medal_tier4'].sum()
print(num1,num2,num3,num4)

In [ ]:
predictors = ['Number of Years Served', 'rating', 'num_medal_tier1','num_medal_tier2','num_medal_tier3','num_medal_tier4']

X_constant = sm.add_constant(data[predictors].astype(float))
y = data['rank_code']

model = sm.Logit(y,X_constant).fit_regularized(method = 'l1', alpha=1.0)
model.summary()

In [ ]:
model = sm.Probit(y,X_constant).fit_regularized(method = 'l1', alpha = 1.0)
model.summary()

In [ ]:
# Define a bootstrap function for custom predictors including medal tiers and rating

def run_bootstrap_custom_logit_model(data, predictors, n_bootstrap=30, verbose=True):
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import statsmodels.api as sm
    import numpy as np
    import pandas as pd

    X = data[predictors].astype(float).reset_index(drop=True)
    y = data['rank_code'].astype(int).reset_index(drop=True)
    coef_names = ['const'] + predictors

    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.5, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None


In [ ]:
predictors = [
    'Number of Years Served', 
    'rating', 
    'num_medal_tier1',
    'num_medal_tier2',
    'num_medal_tier3',
    'num_medal_tier4'
]

result_custom = run_bootstrap_custom_logit_model(data, predictors, n_bootstrap=100)


In [ ]:
predictors = [
    'Number of Years Served', 
    'num_medal_tier1',
    'num_medal_tier2',
    'num_medal_tier3',
    'num_medal_tier4'
]

result_custom = run_bootstrap_custom_logit_model(data, predictors, n_bootstrap=100)


In [ ]:
result_custom["mean_accuracy"]

result_custom["coef_df"]

In [ ]:
coef_df =result_custom["coef_df"].mean()
coef_df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Simulated usage: result_custom["coef_df"] exists from previous bootstrap
# Here we assume result_custom is already defined in the user's session
coef_df = result_custom["coef_df"]

# Calculate summary statistics
coef_mean = coef_df.mean()
coef_std = coef_df.std()
ci_95_lower = coef_mean - 1.96 * coef_std
ci_95_upper = coef_mean + 1.96 * coef_std

# Combine into summary table
coef_summary = pd.DataFrame({
    "Mean": coef_mean,
    "Std": coef_std,
    "95% CI Lower": ci_95_lower,
    "95% CI Upper": ci_95_upper
})

In [ ]:
coef_summary

### Fifth, let's consider manually grouping based PCA.

In [ ]:
# Define the bootstrapped grouped PCA function for both cases (with/without rating)

def run_grouped_pca_bootstrap(
    data,
    medal_groups,
    group_n_components,
    n_bootstrap=30,
    include_rating=True,
    verbose=True
):
    from sklearn.decomposition import PCA
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import statsmodels.api as sm
    import numpy as np
    import pandas as pd

    pcs_all = []

    # Step 1: Run PCA for each group and store results
    for group_name, medals in medal_groups.items():
        n_comp = group_n_components.get(group_name, 1)
        X_group = data[medals].astype(float).fillna(0)

        if X_group.shape[1] == 0 or X_group.var().sum() == 0:
            continue

        pca = PCA(n_components=n_comp)
        group_pcs = pca.fit_transform(X_group)
        pcs_df = pd.DataFrame(group_pcs, columns=[f"{group_name}_PC{i+1}" for i in range(n_comp)])
        pcs_all.append(pcs_df)

        if verbose:
            print(f"{group_name}: {n_comp} component(s) retained")

    # Step 2: Combine all PCs
    all_pcs = pd.concat(pcs_all, axis=1)

    # Step 3: Add other predictors
    base_vars = ['Number of Years Served']
    if include_rating:
        base_vars.append('rating')

    X = pd.concat([data[base_vars].reset_index(drop=True), all_pcs.reset_index(drop=True)], axis=1)
    y = data['rank_code'].astype(int).reset_index(drop=True)
    coef_names = ['const'] + base_vars + list(all_pcs.columns)

    # Step 4: Bootstrap logit
    boot_accuracies = []
    boot_coefs = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        scaler = StandardScaler()
        X_bs_scaled = scaler.fit_transform(X_bs)
        X_bs_scaled = sm.add_constant(X_bs_scaled)

        try:
            model = sm.Logit(y_bs, X_bs_scaled).fit(disp=False)
        except:
            try:
                model = sm.Logit(y_bs, X_bs_scaled).fit_regularized(method='l1', alpha=0.1, disp=False)
                if verbose:
                    print("Fallback to fit_regularized() used.")
            except:
                continue

        try:
            y_pred = (model.predict(X_bs_scaled) > 0.5).astype(int)
            acc = accuracy_score(y_bs, y_pred)
            boot_accuracies.append(acc)
            boot_coefs.append(model.params.values)
        except:
            continue

    if boot_accuracies:
        mean_acc = np.mean(boot_accuracies)
        std_acc = np.std(boot_accuracies)
        coef_df = pd.DataFrame(boot_coefs, columns=coef_names)
        if verbose:
            print(f"Mean Accuracy: {mean_acc:.3f}, Std: {std_acc:.3f}")
        return {
            "mean_accuracy": mean_acc,
            "std_accuracy": std_acc,
            "distribution": boot_accuracies,
            "coef_df": coef_df
        }
    else:
        if verbose:
            print("All bootstrap iterations failed.")
        return None


In [ ]:
tier1 = [
    'Medal of Honor', 'Distinguished Service Cross', 'Navy Cross', 'Silver Star', 'Bronze Star'
]

tier2 = [
    'Distinguished Service Medal', 'Legion of Merit', 'Purple Heart', 'Oak Leaf Cluster',
    'Public Welfare Medal'
]

tier3 = [
    'Civil War Campaign Medal', 'Indian Campaign Medal', 'Mexican Border Service Medal',
    'Philippine Campaign Medal', 'Philippine Congressional Medal',
    'Cuban Pacification Medal', 'Spanish Campaign Medal', 'Spanish War Service Medal',
    'World War I Victory Medal', 'War Merit Cross', 
]

tier4 = [
    'Companion Of The Order Of The Bath (United Kingdom)', 'Croix de Guerre',
    'Czechoslovak War Cross 1918', 'French Legion Of Honor (Commander)',
    'Grand Cordon of the Order of the Sacred Treasure', 'Honorary Knight Commande',
    'Legion of Honor', 'Order Of Leopold Ii (Belgium)', 'Order Of The Black Star (Commander)',
    'Order of La Solidaridad (Panama)', 'Order of Leopold', 'Order of Saints Maurice and Lazarus',
    'Order of St Michael and St George', 'Order of Wen Hu', 'Order of the Bath',
    'Order of the Crown', 'Order of the Dragon of Annam', 'Order of the Rising Sun',
    'Order of the Star of Africa'
]

medal_groups = {
    "tier1": tier1,
    "tier2": tier2,
    "tier3": tier3,
    "tier4": tier4
}

group_n_components = {
    "tier1": 1,
    "tier2": 1,
    "tier3": 1,
    "tier4": 1
}


In [ ]:
result_grouped_with_rating = run_grouped_pca_bootstrap(
    data, medal_groups, group_n_components, include_rating=True
)

In [ ]:
result_grouped_no_rating = run_grouped_pca_bootstrap(
    data, medal_groups, group_n_components, include_rating=False
)

### Sixth, now let's try bootstrap based KRR

In [ ]:
# Re-import required modules after code state reset
def run_bootstrap_krr_model(data, predictors, target_col='rank_code', n_bootstrap=30, alpha=1.0, kernel='rbf', gamma=None, verbose=True):
    from sklearn.kernel_ridge import KernelRidge
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import numpy as np
    import pandas as pd

    X = data[predictors].astype(float).reset_index(drop=True)
    y = data[target_col].astype(int).reset_index(drop=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    boot_accuracies = []
    coef_like_storage = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X_scaled, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        model = KernelRidge(alpha=alpha, kernel=kernel, gamma=gamma)
        model.fit(X_bs, y_bs)

        y_pred_continuous = model.predict(X_bs)
        y_pred_binary = (y_pred_continuous > 0.5).astype(int)
        acc = accuracy_score(y_bs, y_pred_binary)
        boot_accuracies.append(acc)

        # For linear kernel, store "coef-like" from dual + X
        if kernel == 'linear':
            coef_like = X_bs.T @ model.dual_coef_
            coef_like_storage.append(coef_like)

    result = {
        "mean_accuracy": np.mean(boot_accuracies),
        "std_accuracy": np.std(boot_accuracies),
        "distribution": boot_accuracies,
    }

    if kernel == 'linear' and coef_like_storage:
        result["coef_df"] = pd.DataFrame(coef_like_storage, columns=predictors)

    if verbose:
        print(f"Mean Accuracy: {result['mean_accuracy']:.3f}, Std: {result['std_accuracy']:.3f}")

    return result


In [ ]:
predictors = ['Number of Years Served', 'rating', 'num_medal_tier1', 'num_medal_tier2', 'num_medal_tier3', 'num_medal_tier4']

result_krr = run_bootstrap_krr_model(
    data,
    predictors=predictors,
    target_col='rank_code',   # where 0 = Brigadier General, 1 = Major General
    n_bootstrap=30,
    alpha=1.0,
    kernel='rbf',             # or 'linear' if you want interpretable weights
    gamma=0.5                 # optional for RBF
)

In [ ]:
predictors = ['Number of Years Served', 'num_medal_tier1', 'num_medal_tier2', 'num_medal_tier3', 'num_medal_tier4']

result_krr = run_bootstrap_krr_model(
    data,
    predictors=predictors,
    target_col='rank_code',   # where 0 = Brigadier General, 1 = Major General
    n_bootstrap=30,
    alpha=1.0,
    kernel='rbf',             # or 'linear' if you want interpretable weights
    gamma=0.5                 # optional for RBF
)

In [ ]:
medal_columns

In [ ]:
full_predictors = ['Number of Years Served', 'rating'] + medal_columns

# Run the model
result_krr_full = run_bootstrap_krr_model(
    data=data,
    predictors=full_predictors,
    target_col='rank_code',
    n_bootstrap=30,
    alpha=1.0,
    kernel='rbf',
    gamma=0.5
)

In [ ]:
full_predictors = ['Number of Years Served'] + medal_columns

# Run the model
result_krr_full = run_bootstrap_krr_model(
    data=data,
    predictors=full_predictors,
    target_col='rank_code',
    n_bootstrap=30,
    alpha=1.0,
    kernel='rbf',
    gamma=0.5
)

In [ ]:
full_predictors = ['Number of Years Served', 'rating'] + medal_columns

# Run the model
result_krr_full_linear = run_bootstrap_krr_model(
    data=data,
    predictors=full_predictors,
    target_col='rank_code',
    n_bootstrap=30,
    alpha=1.0,
    kernel='linear',
    gamma=0.5
)

In [ ]:
result_krr_full_linear['coef_df']

In [ ]:
full_predictors = ['Number of Years Served'] + medal_columns

# Run the model
result_krr_full_linear = run_bootstrap_krr_model(
    data=data,
    predictors=full_predictors,
    target_col='rank_code',
    n_bootstrap=30,
    alpha=1.0,
    kernel='linear',
    gamma=0.5
)

In [ ]:
result_krr_full_linear['coef_df']

### Seventh, now let's consider Random Forest ands XGBoosting.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Define bootstrap function for Random Forest
def run_bootstrap_random_forest_model(data, predictors, target_col='rank_code', n_bootstrap=30, n_estimators=100, max_depth=None, verbose=True):
    X = data[predictors].astype(float).reset_index(drop=True)
    y = data[target_col].astype(int).reset_index(drop=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    boot_accuracies = []
    feature_importances = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X_scaled, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_bs, y_bs)

        y_pred = model.predict(X_bs)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        feature_importances.append(model.feature_importances_)

    result = {
        "mean_accuracy": np.mean(boot_accuracies),
        "std_accuracy": np.std(boot_accuracies),
        "distribution": boot_accuracies,
        "feature_importances_df": pd.DataFrame(feature_importances, columns=predictors)
    }

    if verbose:
        print(f"Mean Accuracy: {result['mean_accuracy']:.3f}, Std: {result['std_accuracy']:.3f}")

    return result


In [ ]:
full_predictors = ['Number of Years Served', 'rating'] + medal_columns
result_rf = run_bootstrap_random_forest_model(
    data=data,
    predictors=full_predictors,
    n_bootstrap=10,         # reduce from 30 to 10
    n_estimators=50,        # reduce forest size
    max_depth=5,            # cap tree depth
)

In [ ]:
full_predictors = ['Number of Years Served'] + medal_columns
result_rf_norating = run_bootstrap_random_forest_model(
    data=data,
    predictors=full_predictors,
    n_bootstrap=10,         # reduce from 30 to 10
    n_estimators=50,        # reduce forest size
    max_depth=5,            # cap tree depth
)

In [ ]:
from xgboost import XGBClassifier

# Define bootstrap function for XGBoost
def run_bootstrap_xgboost_model(data, predictors, target_col='rank_code', n_bootstrap=30, max_depth=3, learning_rate=0.1, n_estimators=100, verbose=True):
    from sklearn.utils import resample
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import accuracy_score
    import numpy as np
    import pandas as pd

    X = data[predictors].astype(float).reset_index(drop=True)
    y = data[target_col].astype(int).reset_index(drop=True)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    boot_accuracies = []
    feature_importances = []

    for _ in range(n_bootstrap):
        X_bs, y_bs = resample(X_scaled, y, replace=True)
        if len(np.unique(y_bs)) < 2:
            continue

        model = XGBClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            use_label_encoder=False,
            eval_metric='logloss',
            verbosity=0
        )
        model.fit(X_bs, y_bs)

        y_pred = model.predict(X_bs)
        acc = accuracy_score(y_bs, y_pred)
        boot_accuracies.append(acc)
        feature_importances.append(model.feature_importances_)

    result = {
        "mean_accuracy": np.mean(boot_accuracies),
        "std_accuracy": np.std(boot_accuracies),
        "distribution": boot_accuracies,
        "feature_importances_df": pd.DataFrame(feature_importances, columns=predictors)
    }

    if verbose:
        print(f"Mean Accuracy: {result['mean_accuracy']:.3f}, Std: {result['std_accuracy']:.3f}")

    return result


In [ ]:
full_predictors = ['Number of Years Served', 'rating'] + medal_columns
result = run_bootstrap_xgboost_model(
    data=data,
    predictors=full_predictors,
target_col='rank_code', n_bootstrap=30, max_depth=3, learning_rate=0.1, n_estimators=100, verbose=True
)

In [ ]:
full_predictors = ['Number of Years Served'] + medal_columns
result_norating = run_bootstrap_xgboost_model(
    data=data,
    predictors=full_predictors,
target_col='rank_code', n_bootstrap=30, max_depth=3, learning_rate=0.1, n_estimators=100, verbose=True
)

In [ ]:
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

# 1. Prepare the full dataset
X_full = data[full_predictors].astype(float).reset_index(drop=True)
y_full = data['rank_code'].astype(int).reset_index(drop=True)

# 2. Scale features
scaler = StandardScaler()
X_scaled_full = scaler.fit_transform(X_full)

# 3. Fit a single XGBoost model on full data
model_full = XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    verbosity=0
)
model_full.fit(X_scaled_full, y_full)

# 4. Plot PDP for the 'rating' feature
# Get the index of 'rating' in full_predictors list
rating_index = full_predictors.index('rating')

# Plot partial dependence
display = PartialDependenceDisplay.from_estimator(
    model_full,
    X_scaled_full,
    features=[rating_index],
    feature_names=full_predictors,
    kind="average",
    grid_resolution=100
)

plt.title("Partial Dependence of 'rating' on Promotion Probability")
plt.tight_layout()
plt.grid(True)
plt.show()


### Eighth, now let's try the idea from the paper: Causal Inference with Corrupted Data: Measurement Error, Missing Values, Discretization, and Differential Privacy.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt 
from sklearn.decomposition import SparsePCA
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif 
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from fancyimpute import SoftImpute
from econml.dml import DML
from sklearn.model_selection import train_test_split

In [ ]:
print(len(medal_columns))

In [ ]:
X_bin = data[medal_columns].replace(0, np.nan).astype(float)
X_latent = SoftImpute(max_rank = 5).fit_transform(X_bin.values)

pca = PCA(n_components=5)
X_latent_5 = pca.fit_transform(X_latent)  # Now shape is (170, 5)

data_latent = pd.DataFrame(X_latent_5, columns = [f'latent_medal_{i+1}' for i in range(X_latent_5.shape[1])])

In [ ]:
data_latent

In [ ]:
data_combined = pd.concat([data_latent, data[['rank_code', 'rating', 'Number of Years Served']].reset_index(drop=True)], axis = 1)
data_combined

In [ ]:
# Combine covariates
X_covariates = np.column_stack([X_latent_5, data['Number of Years Served']])
T = data['rating'].values
Y = data['rank_code'].values

# Construct design matrix: [T, X]
X_design = sm.add_constant(np.column_stack([T, X_covariates]))

# Run logistic regression
model = sm.Logit(Y, X_design)
result = model.fit()
print(result.summary())


In [ ]:
predictors = ['latent_medal_1', 'latent_medal_2', 'latent_medal_3', 'latent_medal_4', 'latent_medal_5', 'rating', 'Number of Years Served']
X_constant = sm.add_constant(data_combined[predictors].astype(float))
y = data_combined['rank_code']

logit = sm.Logit(y, X_constant)
result = logit.fit(disp=False)

result.summary()

In [ ]:
marginal = result.get_margeff()
print(marginal.summary())

### Nineth, let's consider the Supported Vector Machine method in the process with medal columns.

In [ ]:
predictors = ['Number of Years Served', 'rating'] + medal_columns
target_col = 'rank_code'

X = data[predictors].astype(float)
y = data[target_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linear_svm = SVC(kernel='linear', C=1)
linear_svm.fit(X_train_scaled, y_train)
y_pred_linear = linear_svm.predict(X_test_scaled)
print("🔹 Linear SVM Accuracy:", accuracy_score(y_test, y_pred_linear))
print(classification_report(y_test, y_pred_linear))

rbf_svm = SVC(kernel='rbf', C=1, gamma='scale')
rbf_svm.fit(X_train_scaled, y_train)
y_pred_rbf = rbf_svm.predict(X_test_scaled)
print("🔹 RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))
print(classification_report(y_test, y_pred_rbf))


In [ ]:
# No rating involved
predictors = ['Number of Years Served'] + medal_columns
target_col = 'rank_code'

X = data[predictors].astype(float)
y = data[target_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linear_svm = SVC(kernel='linear', C=1)
linear_svm.fit(X_train_scaled, y_train)
y_pred_linear = linear_svm.predict(X_test_scaled)
print("🔹 Linear SVM Accuracy:", accuracy_score(y_test, y_pred_linear))
print(classification_report(y_test, y_pred_linear))

rbf_svm = SVC(kernel='rbf', C=1, gamma='scale')
rbf_svm.fit(X_train_scaled, y_train)
y_pred_rbf = rbf_svm.predict(X_test_scaled)
print("🔹 RBF SVM Accuracy:", accuracy_score(y_test, y_pred_rbf))
print(classification_report(y_test, y_pred_rbf))
